<a href="https://colab.research.google.com/github/componavt/LLLE-R1900s/blob/main/src/visualization/radial_loan_stars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌐 Radial Loan Profiles: Static Radar Charts per Settlement

Imagine each village as a star — each ray represents a credit category (e.g., "Livestock", "Housing").  
The **length** of the ray = total money borrowed for that purpose.  
The **number of loans** is shown as a label near the tip.

✨ **Features:**
- 📊 One radar chart per settlement
- 📏 Ray length = Σ(amount_rubles) by credit type
- 🔢 Loan count = number label at ray tip
- 🖼️ Rendered directly in Colab (no file saving)
- 🗂️ Based on aggregated loan data

⚙️ Uses `config.env` and loan CSV  
✅ Ready for visual analysis in notebook

In [1]:
# Install compatible versions for static image rendering in Colab
!pip install -q python-dotenv pandas plotly>=6.1.1 kaleido

# Clone the repo if running in Colab (optional — if data not uploaded manually)
import os

if not os.path.exists('LLLE-R1900s'):
    !git clone https://github.com/componavt/LLLE-R1900s.git
    %cd LLLE-R1900s
else:
    %cd LLLE-R1900s

Cloning into 'LLLE-R1900s'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 148 (delta 78), reused 95 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 602.79 KiB | 12.83 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/content/LLLE-R1900s


In [2]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load configuration
load_dotenv('config.env')

# Get paths
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
society_file = 'data/society_settlement.csv'
output_file_name = os.getenv('OUTPUT_CSV_FILE')

if not output_file_name:
    csv_files = [f for f in os.listdir(csv_out_dir) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the output directory.")
    output_file_name = csv_files[0]

csv_path = os.path.join(csv_out_dir, output_file_name)
print(f"Loading loan data from: {csv_path}")

# Load loan data
df_loans = pd.read_csv(csv_path)
print(f"Loaded {len(df_loans)} loan records.")

# Load society/settlement data with coordinates (for reference only)
df_society = pd.read_csv(society_file)
print(f"Loaded {len(df_society)} settlements with coordinates.")

# Aggregate loan data per settlement and credit type
df_agg = df_loans.groupby(['settlement', 'credit_item']).agg(
    total_amount=('amount_rubles', 'sum'),
    total_count=('loan_count', 'sum')
).reset_index()

print("✅ Data aggregated for radar charts.")

Loading loan data from: data/csv_out/loans_s4_i16.csv
Loaded 148 loan records.
Loaded 29 settlements with coordinates.
✅ Data aggregated for radar charts.


In [3]:
import plotly.graph_objects as go

# Get all unique credit items for consistent axes
all_credit_items = sorted(df_agg['credit_item'].unique())
settlements = df_agg['settlement'].unique()

# Track how many charts we actually show
shown_count = 0

for settlement in settlements:
    # Filter data for this settlement
    df_set = df_agg[df_agg['settlement'] == settlement]

    # Build full vectors (including zeros for missing credit types)
    amounts = []
    counts = []
    for item in all_credit_items:
        match = df_set[df_set['credit_item'] == item]
        if not match.empty:
            amt = match['total_amount'].iloc[0]
            cnt = int(match['total_count'].iloc[0])
        else:
            amt = 0
            cnt = 0
        amounts.append(amt)
        counts.append(cnt)

    # Skip if no loans at all
    if sum(amounts) == 0:
        continue

    # Close the loop for radar chart
    theta_labels = all_credit_items + [all_credit_items[0]]
    r_values = amounts + [amounts[0]]
    count_labels = [str(c) if c > 0 else '' for c in counts] + ['']

    # Create figure
    fig = go.Figure()

    # Main filled area
    fig.add_trace(go.Scatterpolar(
        r=r_values,
        theta=theta_labels,
        fill='toself',
        line_color='steelblue',
        opacity=0.6,
        showlegend=False
    ))

    # Loan count markers + text
    fig.add_trace(go.Scatterpolar(
        r=r_values,
        theta=theta_labels,
        mode='markers+text',
        text=count_labels,
        textposition='top center',
        marker=dict(size=8, color='darkred'),
        showlegend=False
    ))

    # Layout
    fig.update_layout(
        title=f"Loan Profile: {settlement}",
        polar=dict(
            radialaxis=dict(visible=True, title="Total Amount (Rubles)"),
            angularaxis=dict(direction="clockwise")
        ),
        showlegend=False,
        width=600,
        height=600,
        font=dict(size=10)
    )

    # Display directly in Colab
    fig.show(renderer="colab")
    shown_count += 1

print(f"\n🎉 Displayed {shown_count} radar charts in Colab.")

/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.





🎉 Displayed 4 radar charts in Colab.
